## 1) Read kafka stream and display

In [ ]:
from kafka import KafkaConsumer
import json

In [ ]:
consumer = KafkaConsumer('text_stram', group_id='1',
                         bootstrap_servers='localhost:9092')
for msg in consumer:
    print(msg)

msg=next(consumer)
print(msg)
msg.key
msg.value
msg.timestamp

## 2) Read Kafka process and save

In [ ]:
import kafka
import sys
import json
from hdfs import InsecureClient
from datetime import datetime

In [ ]:
def main():
    consumer = kafka.KafkaConsumer("alerts", bootstrap_servers="localhost:9092")
    hdfs_client = InsecureClient("http://localhost:50070", user="vagrant")
    hdfs_path = "/user/vagrant/test2/alerts/fires"

    if not hdfs_client.content(hdfs_path, strict=False):
        hdfs_client.makedirs(hdfs_path)
    
    batch = []
    for msg in consumer:
        alert = json.loads(msg.value)
        if alert["keywords"] == "fire" or alert["text"].find("fire") != -1:
            batch.append(alert)
            print(alert)
            if msg["location"] != "":
                log_file.write(msg["location"] + "\n")
        if len(batch) == 10:
            hdfs_client.write(hdfs_path + "/" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + 
                              ".csv", data=json.dumps(batch), encoding = "utf-8")
            batch = []

if __name__ == "__main__":
    main()
    sys.exit()